In [2]:
# 目标网址：http://www.pbc.gov.cn/zhengcehuobisi/125207/125217/125925/index.html

In [ ]:
#-*- coding: utf-8 -*-
import os
import requests
from bs4 import BeautifulSoup
import random
from faker import Factory
import Queue
import threading
 
fake = Factory.create()
luoo_site = 'http://www.luoo.net/music/'
luoo_site_mp3 = 'http://luoo-mp3.kssws.ks-cdn.com/low/luoo/radio%s/%s.mp3'
 
proxy_ips = [    '27.15.236.236'    ] # 替换自己的代理IP
headers = {
    'Connection': 'keep-alive',
    'User-Agent': fake.user_agent()
    }
 
def random_proxies():
    ip_index = random.randint(0, len(proxy_ips)-1)
    res = { 'http': proxy_ips[ip_index] }
    return res
 
def fix_characters(s):
    for c in ['<', '>', ':', '"', '/', '\\\\', '|', '?', '*']:
        s = s.replace(c, '')
    return s
 
 
class LuooSpider(threading.Thread):
    def __init__(self, url, vols, queue=None):
        threading.Thread.__init__(self)
        print '[luoo spider]'
        print '=' * 20
        self.url = url
        self.queue = queue
        self.vol = '1'
        self.vols = vols
 
    def run(self):
        for vol in self.vols:
            self.spider(vol)
        print '\\ncrawl end\\n\\n'
        def spider(self, vol):
        url = luoo_site + vol
        print 'crawling: ' + url + '\\n'
        res = requests.get(url, proxies=random_proxies())
                soup = BeautifulSoup(res.content, 'html.parser')
        title = soup.find('span', attrs={'class': 'vol-title'}).text
        cover = soup.find('img', attrs={'class': 'vol-cover'})['src']
        desc = soup.find('div', attrs={'class': 'vol-desc'})
        track_names = soup.find_all('a', attrs={'class': 'trackname'})
        track_count = len(track_names)
        tracks = []
        for track in track_names:
            _id = str(int(track.text[:2])) if (int(vol) < 12) else track.text[:2]  # 12期前的音乐编号1~9是1位（如：1~9），之后的都是2位 1~9会在左边垫0（如：01~09）
            _name = fix_characters(track.text[4:])
            tracks.append({'id': _id, 'name': _name})
            phases = {
                'phase': vol,                         # 期刊编号
                'title': title,                       # 期刊标题
                 'cover': cover,                      # 期刊封面
                 'desc': desc,                        # 期刊描述
                 'track_count': track_count,          # 节目数
                 'tracks': tracks                     # 节目清单(节目编号，节目名称)
            }
            self.queue.put(phases)
 
 
class LuooDownloader(threading.Thread):
    def __init__(self, url, dist, queue=None):
        threading.Thread.__init__(self)
        self.url = url
        self.queue = queue
        self.dist = dist
        self.__counter = 0       
 
     def run(self):
        while True:
            if self.queue.qsize() <= 0:
                pass
            else:
                phases = self.queue.get()
                self.download(phases)
 
    def download(self, phases):
        for track in phases['tracks']:
            file_url = self.url % (phases['phase'], track['id'])
 
            local_file_dict = '%s/%s' % (self.dist, phases['phase'])
            if not os.path.exists(local_file_dict):
                os.makedirs(local_file_dict)              
 
            local_file = '%s/%s.%s.mp3' % (local_file_dict, track['id'], track['name'])
            if not os.path.isfile(local_file):
                print 'downloading: ' + track['name']
                res = requests.get(file_url, proxies=random_proxies(), headers=headers)
                with open(local_file, 'wb') as f:
                    f.write(res.content)
                    f.close()
                print 'done.\\n'
            else:
                print 'break: ' + track['name']
 
 
if __name__ == '__main__':
    spider_queue = Queue.Queue()
 
    luoo = LuooSpider(luoo_site, vols=['680', '721', '725', '720'],queue=spider_queue)
    luoo.setDaemon(True)
    luoo.start()
 
    downloader_count = 5
    for i in range(downloader_count):
        luoo_download = LuooDownloader(luoo_site_mp3, 'D:/luoo', queue=spider_queue)
        luoo_download.setDaemon(True)
        luoo_download.start()

In [3]:
import requests
from bs4 import BeautifulSoup

In [11]:
target_url = 'http://www.pbc.gov.cn/zhengcehuobisi/125207/125217/125925/index.html'
headers ={
    'Cookie':  'wzwsconfirm=fb606989a6ca9db10446b8d234629f19; wzwstemplate=Mw==; ccpassport=ec2b469c4aca05139b5949dc3c712220; wzwschallenge=-1; wzwsvtime=1532409555',
    'User-Agnet': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'
}
res = requests.get(target_url, headers=headers)
soup = BeautifulSoup(res.content, 'html.parser')
target_href = soup.find('a')

In [47]:
import re
# xpath: //*[@id="17105"]/div[2]/div[1]/table/tbody/tr[2]/td/table[1]/tbody/tr/td[2]/font/a
target_all_a = soup.find_all('a')
target_text = '中国外汇交易中心受权公布人民币汇率中间价公告">'
for item in target_all_a:
    item = str(item)
    try:
        target_href_pos = re.search(target_text,item).span()
        if target_href_pos:
            target_date = item[107:target_href_pos[0]]
            target_href = item[9:target_href_pos[0]-53]
            print(target_date)
            print(target_href)
    except:
        continue

2018年7月24日
/zhengcehuobisi/125207/125217/125925/3584593/index.html
2018年7月23日
/zhengcehuobisi/125207/125217/125925/3582807/index.html
2018年7月20日
/zhengcehuobisi/125207/125217/125925/3581885/index.html
2018年7月19日
/zhengcehuobisi/125207/125217/125925/3581280/index.html
2018年7月18日
/zhengcehuobisi/125207/125217/125925/3580608/index.html
2018年7月17日
/zhengcehuobisi/125207/125217/125925/3579553/index.html
2018年7月16日
/zhengcehuobisi/125207/125217/125925/3578267/index.html
2018年7月13日
/zhengcehuobisi/125207/125217/125925/3577127/index.html
2018年7月12日
/zhengcehuobisi/125207/125217/125925/3576374/index.html
2018年7月11日
/zhengcehuobisi/125207/125217/125925/3576024/index.html
2018年7月10日
/zhengcehuobisi/125207/125217/125925/3574980/index.html
2018年7月9日
/zhengcehuobisi/125207/125217/125925/3573733/index.htm
2018年7月6日
/zhengcehuobisi/125207/125217/125925/3572645/index.htm
2018年7月5日
/zhengcehuobisi/125207/125217/125925/3572147/index.htm
2018年7月4日
/zhengcehuobisi/125207/125217/125925/3571407/index.htm
201

In [51]:
target_href = '/zhengcehuobisi/125207/125217/125925/3584593/index.html'
new_url = 'http://www.pbc.gov.cn' + target_href

headers ={
    'Cookie':  'wzwsconfirm=e79f35eb1e05969d4aac70d6a38c9b53; wzwstemplate=Mg==; ccpassport=ec2b469c4aca05139b5949dc3c712220; wzwschallenge=-1; wzwsvtime=1532412429',
    'User-Agnet': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'
}
res = requests.get(new_url, headers=headers)
soup = BeautifulSoup(res.content, 'html.parser')

In [54]:
source_text = soup.find_all('p')[0]
print(source_text)

<p>中国人民银行授权中国外汇交易中心公布，2018年7月24日银行间外汇市场人民币汇率中间价为：1美元对人民币6.7891元，1欧元对人民币7.9387元，100日元对人民币6.0963元，1港元对人民币0.86513元，1英镑对人民币8.8977元，1澳大利亚元对人民币5.0116元，1新西兰元对人民币4.6063元，1新加坡元对人民币4.9741元，1瑞士法郎对人民币6.8395元，1加拿大元对人民币5.1553元，人民币1元对0.59813林吉特，人民币1元对9.2705俄罗斯卢布, 人民币1元对1.9816南非兰特，人民币1元对166.82韩元，人民币1元对0.54075阿联酋迪拉姆，人民币1元对0.55214沙特里亚尔，人民币1元对41.0841匈牙利福林，人民币1元对0.54489波兰兹罗提，人民币1元对0.9382丹麦克朗，人民币1元对1.3048瑞典克朗，人民币1元对1.2069挪威克朗，人民币1元对0.69735土耳其里拉，人民币1元对2.7862墨西哥比索，人民币1元对4.9200泰铢。</p>
